In [2]:
import csv, sqlite3
import pandas as pd

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [3]:
%load_ext sql
%sql sqlite:///FinalDB.db

In [5]:
df = pd.read_csv(r"C:\Users\Javi\Documents\GitHub\Chicago-crime-datasets\data\ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


78

In [6]:

df = pd.read_csv(r"C:\Users\Javi\Documents\GitHub\Chicago-crime-datasets\data\ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")


533

In [18]:
df = pd.read_csv(r"data\ChicagoPublicSchools.csv")

In [23]:
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False)

566

In [24]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


## Problems    
Now write and execute SQL queries to solve assignment problems

### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [62]:
%%sql

SELECT COUNT(*) FROM CENSUS_DATA

 * sqlite:///FinalDB.db
Done.


COUNT(*)
78


### Problem 2
List community area names and numbers with per capita income less than 11000.

In [27]:
%%sql 

SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


### Problem 3
List all case numbers for crimes involving minors(children are not considered minors for the purposes of crime analysis) 

In [29]:
%%sql 

SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


### Problem 4
List all kidnapping crimes involving a child?

### Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [48]:
%sql SELECT * FROM CENSUS_DATA LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0


In [39]:
%sql SELECT * FROM CHICAGO_CRIME_DATA LIMIT 1

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)"


In [57]:
%%sql 

SELECT * FROM CHICAGO_PUBLIC_SCHOOLS_DATA 
ORDER BY COMMUNITY_AREA_NUMBER
LIMIT 10

 * sqlite:///FinalDB.db
Done.


School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
609918,Eugene Field Elementary School,ES,7019 N Ashland Ave,Chicago,IL,60626,(773) 534-2030,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609918.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Average,42.0,Weak,33,Average,47.0,Weak,37.0,Weak,21,Very Weak,8,Weak,44,Average,53,96.80%,10.0,96.70%,100.00%,NDA,NDA,23.9,24.3,49.4,51.4,48.4,36.4,70.3,53.2,7.4,13,9.8,7.9,-0.2,0.1,Yellow,Yellow,45.6,20,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,375,32,NDA,1164539.561,1946810.338,42.00966462,-87.66994862,1,ROGERS PARK,49,24,"(42.00966462, -87.66994862)"
609865,Jordan Elementary Community School,ES,7414 N Wolcott Ave,Chicago,IL,60626,(773) 534-2220,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609865.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61.0,Weak,31,Average,57.0,Average,49.0,Weak,17,Very Weak,15,Average,52,Average,51,95.60%,16.4,95.10%,100.00%,60.1,42,36,21.1,41.4,54.1,26.3,19.1,43.2,34.7,4.8,15.9,16.9,7.1,0.3,-1.2,Yellow,Red,37.7,23.1,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,753,32,NDA,1162385.224,1949501.716,42.01709536,-87.67779928,1,ROGERS PARK,49,24,"(42.01709536, -87.67779928)"
610022,Joyce Kilmer Elementary School,ES,6700 N Greenview Ave,Chicago,IL,60626,(773) 534-2115,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610022.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,NDA,None,Average,43,NDA,None,NDA,None,Weak,43,Average,42,Average,49,Average,51,96.00%,15.6,96.90%,100.00%,70,51.2,40.5,28.6,53.4,61.2,39.4,33.7,57.8,52.8,22.7,27.3,22.8,15.0,-0.2,-0.6,Yellow,Yellow,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,886,32,NDA,1165015.365,1944631.682,42.00367622,-87.66826026,1,ROGERS PARK,40,24,"(42.00367622, -87.66826026)"
610284,New Field Elementary School,ES,1707 W Morse Ave,Chicago,IL,60626,(773) 534-2760,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610284.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,NDA,None,NDA,NDA,NDA,None,NDA,None,Weak,NDA,NDA,NDA,Average,51,Strong,56,96.20%,3.1,95.60%,99.00%,74.2,52.7,32.1,25.2,40.4,55.9,NDA,NDA,NDA,NDA,NDA,NDA,23.2,10.9,None,None,NDA,NDA,NDA,NDA,NDA,NDA,NDA,ND

In [64]:
%%sql 

SELECT * FROM CHICAGO_CRIME_DATA CCD, CHICAGO_PUBLIC_SCHOOLS_DATA CPD
    WHERE CCD.COMMUNITY_AREA_NUMBER = CPD.COMMUNITY_AREA_NUMBER
    LIMIT 5


 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE_1,Y_COORDINATE_1,Latitude,Longitude,COMMUNITY_AREA_NUMBER_1,COMMUNITY_AREA_NAME,Ward,Police_District,Location
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)",609715,Thomas Kelly High School,HS,4136 S California Ave,Chicago,IL,60632,(773) 535-4900,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609715.pdf,Southwest Side High School Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Weak,36.0,Average,53,Weak,36.0,Weak,35.0,Weak,61,Average,57,NDA,NDA,NDA,NDA,89.30%,5.8,95.30%,99.70%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,13.4,13.4,15.1,14.6,1.2,17.1,2,22.1,67.4,46.4,2883,39,65.6,1158337.786,1877175.576,41.81871101,-87.69467483,58,BRIGHTON PARK,14,9,"(41.81871101, -87.69467483)"
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)",609829,John C Burroughs Elementary School,ES,3542 S Washtenaw Ave,Chicago,IL,60632,(773) 535-7226,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609829.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 2,Yes,Average,50.0,Average,55,Weak,39.0,Weak,38.0,Weak,71,Average,53,Average,50,Weak,45,95.60%,0.2,96.10%,100.00%,67.3,27.9,47.7,43.4,55.9,45.1,44.2,57.4,51.9,64.3,25,15.9,25.9,16.0,0.1,-0.1,Yellow,Yellow,42.2,21.1,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,510,39,NDA,1158904.877,1880879.379,41.8288631,-87.69249318,58,BRIGHTON PARK,12,9,"(41.8288631, -87.69249318)"
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)",609876,Nathan S Davis Elementary School,ES,3014 W 39th Pl,Chicago,IL,60632,(773) 535-4540,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609876.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,No,Average,43.0,NDA,NDA,Average,41.0,Average,46.0,Weak,NDA,NDA,NDA,Average,48,Average,50,95.10%,2.4,95.90%,9

### Problem 1
Find the total number of crimes recorded in the CRIME table.

### Problem 1
Find the total number of crimes recorded in the CRIME table.

### Problem 1
Find the total number of crimes recorded in the CRIME table.

### Problem 1
Find the total number of crimes recorded in the CRIME table.

### Problem 1
Find the total number of crimes recorded in the CRIME table.